In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


Convert all columns to numeric values

In [4]:
df.loc[df['Sex'] == 'M', 'Sex'] = 0
df.loc[df['Sex'] == 'F', 'Sex'] = 1

In [5]:
df.loc[df['ChestPainType'] == 'TA', 'ChestPainType'] = 3
df.loc[df['ChestPainType'] == 'ATA', 'ChestPainType'] = 2
df.loc[df['ChestPainType'] == 'NAP', 'ChestPainType'] = 1
df.loc[df['ChestPainType'] == 'ASY', 'ChestPainType'] = 0

In [6]:
df.loc[df['RestingECG'] == 'ST', 'RestingECG'] = 2
df.loc[df['RestingECG'] == 'LVH', 'RestingECG'] = 1
df.loc[df['RestingECG'] == 'Normal', 'RestingECG'] = 0

In [7]:
df.loc[df['ExerciseAngina'] == 'Y', 'ExerciseAngina'] = 1
df.loc[df['ExerciseAngina'] == 'N', 'ExerciseAngina'] = 0

In [8]:
df.loc[df['ST_Slope'] == 'Down', 'ST_Slope'] = 2
df.loc[df['ST_Slope'] == 'Flat', 'ST_Slope'] = 1
df.loc[df['ST_Slope'] == 'Up', 'ST_Slope'] = 0

In [9]:
# coerce all data types to numerics
df = df.apply(pd.to_numeric, errors='coerce')

In [10]:
df.dtypes

Age                 int64
Sex                 int64
ChestPainType       int64
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG          int64
MaxHR               int64
ExerciseAngina      int64
Oldpeak           float64
ST_Slope            int64
HeartDisease        int64
dtype: object

Separate and normalize the data

In [11]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
x = df.drop(columns=['HeartDisease'])
y = df['HeartDisease']

In [13]:
# Split training and testing data
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=83)

In [14]:
xtrain = xtrain.reset_index(drop=True)
xtest = xtest.reset_index(drop=True)
ytrain = ytrain.reset_index(drop=True)
ytest = ytest.reset_index(drop=True)

In [15]:
# Feature scaling - convert all features to z-scores
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.fit_transform(xtest)

Train the model

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras

In [17]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [18]:
model = Sequential(
    [
        Dense(units=11, activation='sigmoid', name='input'),
        Dense(units=5, activation='sigmoid', name='h1'),
        Dense(units=1, activation='sigmoid', name='out')
    ]
)

In [19]:
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy', 'precision', 'recall'])

In [20]:
model.fit(x=xtrain, y=ytrain, epochs=1000, batch_size=10)

Epoch 1/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5457 - loss: 0.6982 - precision: 0.5457 - recall: 1.0000 
Epoch 2/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5367 - loss: 0.6792 - precision: 0.5367 - recall: 1.0000 
Epoch 3/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5407 - loss: 0.6572 - precision: 0.5386 - recall: 0.9989 
Epoch 4/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7130 - loss: 0.6260 - precision: 0.6598 - recall: 0.9836 
Epoch 5/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7993 - loss: 0.5860 - precision: 0.7593 - recall: 0.9255 
Epoch 6/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8408 - loss: 0.5375 - precision: 0.8291 - recall: 0.9068 
Epoch 7/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8549 - loss: 0.5020 - precision: 0.8321 - recall: 0.9077 
Epoch 8/1000
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8306 - loss: 0.4956 - precision: 0.8224 - recall: 0.8558 
Epoch 9/

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (Dense)                        │ (None, 11)                  │             132 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ h1 (Dense)                           │ (None, 5)                   │              60 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out (Dense)                          │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 596 (2.33 KB)

 Trainable params: 198 (792.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 398 (1.56 KB)

In [31]:
# Evaluate using the test data
loss, acurracy, precision, recall = model.evaluate(xtest, ytest)
print(f'{loss=}, {acurracy=}, {precision=}, {recall=}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9007 - loss: 0.3663 - precision: 0.8912 - recall: 0.9442
loss=0.4482140839099884, acurracy=0.8695651888847351, precision=0.8623853325843811, recall=0.9126213788986206
